In [1]:
import pandas as pd
import numpy as np
import difflib as df
import spacy
import glob
from spacy.util import compile_infix_regex
from spacy.tokenizer import Tokenizer
# from allennlp.predictors.predictor import Predictor

In [2]:
nlp = spacy.load('en_core_web_sm')
# model_url = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz"
# predictor = Predictor.from_path(model_url)

In [3]:
# def custom_tokenizer(nlp):
#     inf = list(nlp.Defaults.infixes)             
#     inf[6] = inf[6].replace("0-9", "")    
#     inf = tuple(inf)                               
#     infix_re = compile_infix_regex(inf)

#     return Tokenizer(nlp.vocab, prefix_search=nlp.tokenizer.prefix_search,
#                                 suffix_search=nlp.tokenizer.suffix_search,
#                                 infix_finditer=infix_re.finditer,
#                                 token_match=nlp.tokenizer.token_match,
#                                 rules=nlp.Defaults.tokenizer_exceptions)

In [4]:
# nlp.tokenizer = custom_tokenizer(nlp)

# Read Files

In [5]:
ACE2004_54_truth = pd.read_csv("../Data/relex_processed_data/relex/ACE2004/ground_truth/20000715_AFP_ARB.0054.eng.csv", sep = "\t")
ACE2004_54_tagged = pd.read_csv("../Data/relex_processed_data/relex/ACE2004/tagged_tokens/20000715_AFP_ARB.0054.eng.csv", sep = "\t")

In [6]:
ACE2004_54_text = open("../Data/relex_processed_data/relex/ACE2004/raw_text/20000715_AFP_ARB.0054.eng.txt", "r", encoding="utf8")

In [7]:
ACE2004_54_text_list = ACE2004_54_text.readlines()

In [8]:
# cleaning raw text file
ACE2004_54_text_list = [sentence for sentence in ACE2004_54_text_list if sentence != '\n']
ACE2004_54_text_list = [sentence.replace(' \n', '') for sentence in ACE2004_54_text_list]

In [9]:
ACE2004_54_text_list

['Harare 7-15 (AFP)-',
 "Zimbabwe's President Robert Mugabe announced to journalists this evening Saturday that he had formed a new government, following legislative elections last June, with no opposition members included.",
 'The government, which was formed following legislative elections held last month and which the ruling party won by a slim majority, includes 19 ministers all of whom are members of the Zimbabwe African National Union - Patriotic Front.',
 'The new government consists of 17 ministers and 2 undersecretaries.',
 'The list, announced by President Mugabe at a press conference, reveals that the new government includes 10 new ministers selected from public figures and given ministerial posts for the first time, including a young minister named Simba Makoni appointed as head of the Ministry of Finance and Economic Development under circumstances where the country is witnessing an unprecedented economic crisis.',
 'Elsewhere, Stan Modinge retains his appointment as Forei

In [10]:
# def create_tagged_dict(tagged_token):
#     tagged_dict = {} # creating a data dict using the tagged dataframe
#     sent_count = 0
#     word_count = 0
#     error_flag = 0
#     for index, row in tagged_token.iterrows():
        
#         if row["text"] == "'":
#             error_flag += 1
#             if error_flag < 2:
#                 key = str(sent_count) + "_" + str(word_count)+ "_"  + str(row["text"])
#                 tagged_dict[key] = [row["pos"], row["tag"], row["dep"], row["entity type"]]
#                 word_count += 1
#                 if row["pos"] == "SPACE":
#                     sent_count += 1
#                     word_count = 0
            
#         elif row["text"] != "'":
#             error_flag = 0
#             key = str(sent_count) + "_" + str(word_count)+ "_"  + str(row["text"])
#             tagged_dict[key] = [row["pos"], row["tag"], row["dep"], row["entity type"]]
#             word_count += 1
#             if row["pos"] == "SPACE":
#                 sent_count += 1
#                 word_count = 0
            
#     return tagged_dict

In [11]:
def create_tagged_dict(tagged_token):
    tagged_dict = {} # creating a data dict using the tagged dataframe
    sent_count = 0
    word_count = 0
    error_flag = 0
    for index, row in tagged_token.iterrows():
        
        key = str(sent_count) + "_" + str(word_count)
        tagged_dict[key] = [row["pos"], row["tag"], row["dep"], row["entity type"], row["entity mention ID"], str(row["text"])]
        word_count += 1
        if row["pos"] == "SPACE":
            sent_count += 1
            word_count = 0
            
    return tagged_dict

In [12]:
tagged_dict = create_tagged_dict(ACE2004_54_tagged)

In [13]:
tagged_dict

{'0_0': ['PROPN', 'NNP', 'nmod', 'GPE', '19-1', 'Harare'],
 '0_1': ['NUM', 'CD', 'nummod', nan, nan, '7'],
 '0_2': ['SYM', 'SYM', 'punct', nan, nan, '-'],
 '0_3': ['NUM', 'CD', 'prep', nan, nan, '15'],
 '0_4': ['PUNCT', '-LRB-', 'punct', nan, nan, '('],
 '0_5': ['PROPN', 'NNP', 'intj', 'ORG', '20-29', 'AFP)-'],
 '0_6': ['SPACE', '_SP', nan, nan, nan, ' '],
 '1_0': ['PROPN', 'NNP', 'poss', 'GPE', '21-30', 'Zimbabwe'],
 '1_1': ['PART', 'POS', 'case', nan, nan, "'s"],
 '1_2': ['PROPN', 'NNP', 'compound', 'PER', '22-34', 'President'],
 '1_3': ['PROPN', 'NNP', 'compound', 'PER', '22-35', 'Robert'],
 '1_4': ['PROPN', 'NNP', 'nsubj', 'PER', '22-35', 'Mugabe'],
 '1_5': ['VERB', 'VBD', 'ROOT', nan, nan, 'announced'],
 '1_6': ['ADP', 'IN', 'prep', nan, nan, 'to'],
 '1_7': ['NOUN', 'NNS', 'pobj', 'PER', '25-39', 'journalists'],
 '1_8': ['DET', 'DT', 'det', nan, nan, 'this'],
 '1_9': ['NOUN', 'NN', 'npadvmod', nan, nan, 'evening'],
 '1_10': ['PROPN', 'NNP', 'npadvmod', nan, nan, 'Saturday'],
 '1_1

# Sentence Entity

In [14]:
def sentence_entity(tagged_dict, sentence_index, sentence_nlp, enhanced_bool = False):
    
    entities = []
    entity = ""
    entity_index = ""
    prev_token = ""
    prev_token_index = ""
    indices = []
    entities_types = []
    
    
    if enhanced_bool == True:
        for index, token in enumerate(sentence_nlp):

            if token.text == "''":
                token_text = "'"
            else:
                token_text = token.text

            key = str(sentence_index) + "_" + str(index)

            pos = tagged_dict[key][0]
            dep = str(tagged_dict[key][2])
            entity_type = tagged_dict[key][3]

            if not pd.isnull(entity_type):
                entity = entity + " " + token_text
                if entity_index != "":
                    entity_index = entity_index + "_" + str(index)
                else:
                    entity_index = str(index)

            elif pd.isnull(entity_type) and (dep == "compound" or dep.endswith("mod")):
                prev_token = prev_token + " " + token_text
                if prev_token_index != "":
                    prev_token_index = prev_token_index + "_" + str(index)
                else:
                    prev_token_index = str(index)

            elif pd.isnull(entity_type) and dep != "compound" and dep.endswith("mod") == False and entity == "":
                prev_token = ""
                prev_token_index = ""

            entity = entity.strip()

            if pd.isnull(entity_type) and entity != "":

                if prev_token != "":
                    entity = prev_token + " " + entity
                    entity_index = prev_token_index + "_" + entity_index

                entities.append(entity.strip())
                indices.append(entity_index)
                entities_types.append(entity_type.strip())
                entity = ""
                entity_index = ""
                prev_token = ""
                prev_token_index = ""
    else:
        for key in tagged_dict.keys():
            
            key_split = key.split("_")
            sentence_id = key_split[0]
            key_id = key_split[1]
            
            if str(sentence_id) == str(sentence_index):
                entity_type = tagged_dict[key][3]
                entity_id = tagged_dict[key][4]
                
                if not pd.isnull(entity_type):
                    
                    if len(indices) != 0 and indices[-1][0] == entity_id:
                        entities[-1] = entities[-1] + " " + tagged_dict[key][5]
                        
                    else:
                        entities.append(tagged_dict[key][5])
                        indices.append([entity_id, key_id])
                        entities_types.append(entity_type.strip())
                    
    return entities, indices, entities_types

In [15]:
def extract_root(tagged_dict, sentence_index, sentence_nlp):
    
    sent_roots = []
    sent_roots_index = []
    
    for index, token in enumerate(sentence_nlp):
        
        if token.text == "''":
            token_text = "'"
        else:
            token_text = token.text
            
        key = str(sentence_index) + "_" + str(index)
        dep = tagged_dict[key][2]
        
        if dep == "ROOT":
            sent_roots.append(token.text)
            sent_roots_index.append(index)
            
    return sent_roots, sent_roots_index
        

In [16]:
def filter_sentences(entity_list):
    
    for index, sentence_entity in enumerate(entity_list):
        if len(sentence_entity[2][0]) == 0:
            entity_list.pop(index)
            
    return entity_list

In [17]:
def resolve_corefernce(sentence, sentence_nlp, entites, indices):
    prediction = predictor.predict(document=sentence)
    clusters = prediction['clusters']
    
    if (len(clusters)) != 0:
        for cluster in clusters:
            if cluster[1][0] == cluster[1][1]:
                index = cluster[1][1]
                if str(index) in indices:
                    resolve_index = indices.index(str(index))
                    resolution_index = cluster[0]
                    entites[resolve_index] = sentence_nlp[resolution_index[0] : resolution_index[1] + 1].text
            
    return entites

In [18]:
sentences_entity_list = []

for index, sentence in enumerate(ACE2004_54_text_list):
    sentence_nlp = nlp(sentence)
    entities, indices, entities_types = sentence_entity(tagged_dict, index, sentence_nlp, False)
    # entities = resolve_corefernce(sentence, sentence_nlp, entities, indices) #resolving coreferences
    # sent_roots, sent_roots_index = extract_root(tagged_dict, index, sentence_nlp)
    sentences_entity_list.append([index, [entities, indices, entities_types]])
    
# sentences_entity_list = filter_sentences(sentences_entity_list)

In [19]:
for index, entity_list in enumerate(sentences_entity_list):
    print(entity_list[0])
    print(ACE2004_54_text_list[entity_list[0]])
    print(entity_list[1])
    # print(entity_list[2])
    print("\n")

0
Harare 7-15 (AFP)-
[['Harare', 'AFP)-'], [['19-1', '0'], ['20-29', '5']], ['GPE', 'ORG']]


1
Zimbabwe's President Robert Mugabe announced to journalists this evening Saturday that he had formed a new government, following legislative elections last June, with no opposition members included.
[['Zimbabwe', 'President', 'Robert Mugabe', 'journalists', 'he', 'government', 'members'], [['21-30', '0'], ['22-34', '2'], ['22-35', '3'], ['25-39', '7'], ['22-36', '12'], ['26-40', '17'], ['27-41', '28']], ['GPE', 'PER', 'PER', 'PER', 'PER', 'ORG', 'PER']]


2
The government, which was formed following legislative elections held last month and which the ruling party won by a slim majority, includes 19 ministers all of whom are members of the Zimbabwe African National Union - Patriotic Front.
[['government', 'which', 'party', 'ministers', 'all', 'whom', 'members', 'Zimbabwe African National Union - Patriotic Front'], [['26-42', '1'], ['26-43', '3'], ['28-44', '16'], ['1-3', '25'], ['1-4', '26'],

In [20]:
def find_enhanced_relation(tagged_dict, sentence_index, entity_list, sentence):
    
    entities_relations = []
    
    sentence_entities = entity_list[1]
    
    sentence_nlp = nlp(sentence)
    
    entities_text = sentence_entities[0]
    entities_text_indices = sentence_entities[1]
    entities_len = len(entities_text)
    
    for index, entity_text in enumerate(entities_text):
        
        entity1 = ""
        entity1_index = ""
        entity2 = ""
        entity2_index = ""
        
        entity_pair_relation = ""
        entity_pair_relation_pos_pattern = ""
        
        if index == entities_len - 1:
            continue
        else:
            
            entity1 = entity_text
            entity1_index = entities_indices[index][1]
            entity1_id = entities_indices[index][0]
            
            entity2 = entities_text[index + 1]
            entity2_index = entities_indices[index + 1][1]
            entity2_id = entities_indices[index + 1][0]
            
            for i in range(int(entity1_index) + 1, int(entity2_index)):
                
                if sentence_nlp[i].text == "''":
                    token_text = "'"
                else:
                    token_text = sentence_nlp[i].text
                
                entity_pair_relation = entity_pair_relation + " " + token_text
                
                if entity_pair_relation_pos_pattern != "":
                    entity_pair_relation_pos_pattern = entity_pair_relation_pos_pattern + "-" + tagged_dict[str(sentence_index) + "_" + str(i)][0]
                else:
                    entity_pair_relation_pos_pattern = tagged_dict[str(sentence_index) + "_" + str(i)][0]
                    
        entity_pair_relation = entity1 + " " + entity_pair_relation.strip() + " " + entity2 
        entities_relations.append([[entity1, entity2],[entity_pair_relation, entity_pair_relation_pos_pattern]])
    
    return entities_relations

In [21]:
def find_relation(tagged_dict, sentence_index, entity_list):
    
    entities_relations = []
    
    sentence_entities = entity_list[1]
    
    entities_text = sentence_entities[0]
    entities_indices = sentence_entities[1]
    entities_type = sentence_entities[2]
    entities_len = len(entities_text)
    
    for index, entity_text in enumerate(entities_text):
        
        entity1 = ""
        entity1_index = ""
        entity2 = ""
        entity2_index = ""
        
        entity_pair_relation = ""
        entity_pair_relation_pos_pattern = ""
        
        if index == entities_len - 1:
            continue
        else:
            
            entity1 = entity_text
            entity1_index = entities_indices[index][1]
            entity1_id = entities_indices[index][0]
            entity1_type = entities_type[index]
            
            entity2 = entities_text[index + 1]
            entity2_index = entities_indices[index + 1][1]
            entity2_id = entities_indices[index + 1][0]
            entity2_type = entities_type[index + 1]
            
            for i in range(int(entity1_index) + 1, int(entity2_index)):
                
                token_text = tagged_dict[str(sentence_index) + "_" + str(i)][5]
                
                entity_pair_relation = entity_pair_relation + " " + token_text
                
                if entity_pair_relation_pos_pattern != "":
                    entity_pair_relation_pos_pattern = entity_pair_relation_pos_pattern + "-" + tagged_dict[str(sentence_index) + "_" + str(i)][0]
                else:
                    entity_pair_relation_pos_pattern = tagged_dict[str(sentence_index) + "_" + str(i)][0]
                    
        entity_pair_relation = entity1 + " " + entity_pair_relation.strip() + " " + entity2
        entity_pair_type = entity1_type + "-" + entity2_type
        entities_relations.append([[entity1, entity2], [entity1_id, entity2_id, entity_pair_type], [entity_pair_relation, entity_pair_relation_pos_pattern]])
    
    return entities_relations

In [22]:
for index, entity_list in enumerate(sentences_entity_list):
    entities_relations = find_relation(tagged_dict, entity_list[0], entity_list)
    print(ACE2004_54_text_list[entity_list[0]])
    print(entity_list[1])
    print("=====")
    for entity_relation in entities_relations:
        print(entity_relation)
    print("=====")
    print("\n")

Harare 7-15 (AFP)-
[['Harare', 'AFP)-'], [['19-1', '0'], ['20-29', '5']], ['GPE', 'ORG']]
=====
[['Harare', 'AFP)-'], ['19-1', '20-29', 'GPE-ORG'], ['Harare 7 - 15 ( AFP)-', 'NUM-SYM-NUM-PUNCT']]
=====


Zimbabwe's President Robert Mugabe announced to journalists this evening Saturday that he had formed a new government, following legislative elections last June, with no opposition members included.
[['Zimbabwe', 'President', 'Robert Mugabe', 'journalists', 'he', 'government', 'members'], [['21-30', '0'], ['22-34', '2'], ['22-35', '3'], ['25-39', '7'], ['22-36', '12'], ['26-40', '17'], ['27-41', '28']], ['GPE', 'PER', 'PER', 'PER', 'PER', 'ORG', 'PER']]
=====
[['Zimbabwe', 'President'], ['21-30', '22-34', 'GPE-PER'], ["Zimbabwe 's President", 'PART']]
[['President', 'Robert Mugabe'], ['22-34', '22-35', 'PER-PER'], ['President  Robert Mugabe', '']]
[['Robert Mugabe', 'journalists'], ['22-35', '25-39', 'PER-PER'], ['Robert Mugabe Mugabe announced to journalists', 'PROPN-VERB-ADP']]
[['jo

## Snowball for POS Pattern

In [23]:
import os
import glob
import copy
from random import shuffle
import operator
from pandas.errors import EmptyDataError
from collections import Counter

In [24]:
ACE2004_file_dir = "../Data/relex_processed_data/relex/ACE2004/ground_truth/*.csv"
ACE2004_file_dir_files = glob.glob(ACE2004_file_dir)

In [25]:
def cal_pos_pattern_count(snowball_index, pos_pattern_list):
    empty_ct = 0
    sum_ct = 0
    sum_ct = len(pos_pattern_list)
    for index, pos_pattern in enumerate(pos_pattern_list):
        if pos_pattern in snowball_index:
            snowball_index[pos_pattern] += 1
            # sum_ct =sum_ct + 1
        elif pos_pattern != pos_pattern:
            empty_ct += 1
            pass
        else:
            snowball_index[pos_pattern] = 1
            # sum_ct =sum_ct + 1

    return snowball_index, empty_ct, sum_ct

In [26]:
def generate_snowball(files):
    
    snowball_index = {}

    for file in files:
        try:
            file_tagged_truth = pd.read_csv(file, sep = "\t")
            if len(file_tagged_truth) != 0:
                pos_pattern_list = file_tagged_truth.pos_pattern.tolist()
                snowball_index = cal_pos_pattern_count(snowball_index, pos_pattern_list)
        except EmptyDataError:
            print(f"No columns to parse from file {file}")
            
    return snowball_index

In [27]:
def remove_out_rel_type(all_rel, cutoff):
    
    rel_type_out = []
    
    dict_count = Counter(all_rel)
    val_sum = 0
    for value in dict_count.values():
        val_sum += value
        
    for key, value in dict_count.items():
        rel_dist = (value/val_sum)*100
        if rel_dist < cutoff:
            rel_type_out.append(key)
    
    return rel_type_out

In [28]:
def generate_unique_rel(files, cutoff):
    
    unique_rel = set()
    
    unique_rel_dict = {}
    unique_rel_set = {}
    
    all_rel = []
    
    for file in files:
        try:
            file_tagged_truth = pd.read_csv(file, sep = "\t")
            if len(file_tagged_truth) != 0:
                rel_type = file_tagged_truth["rel type"].tolist()
                for rel in rel_type:
                    all_rel.append(rel)
                rel_type = set(rel_type)
                unique_rel.update(rel_type)
        except EmptyDataError:
            print(f"No columns to parse from file {file}")
    
    rel_type_out = remove_out_rel_type(all_rel, cutoff)
    
    unique_rel.difference_update(set(rel_type_out))
    
    for rel in unique_rel:
        unique_rel_dict[rel] = {}
        unique_rel_set[rel] = set()
    
    return unique_rel_dict, unique_rel_set, rel_type_out

In [29]:
def sort_entity_ids(entity_ids):
    
    for ids in entity_ids:
        ids.sort()
        
    return entity_ids

## Generate Rel Snowball function

In [30]:
def generate_rel_snowball(files, cutoff):
    
    snowball_index = {}
    
    snowball_entity = {}
    
    snowball_index_rel, snowball_pos_ent_type, rel_type_out = generate_unique_rel(files, cutoff)
    
    list_empty_ct = []
    
    list_sum_ct = []
    
    for rel in snowball_index_rel.keys():
        for file in files:
            try:
                file_tagged_truth = pd.read_csv(file, sep = "\t")
                # print(file)
                file_tagged_truth = file_tagged_truth[file_tagged_truth["pos_pattern"].notna()]
                if len(file_tagged_truth) != 0:
                    pos_pattern_list = file_tagged_truth.pos_pattern[file_tagged_truth['rel type'] == rel].tolist()
                    
                    entities_type = file_tagged_truth[["entity 1 mention ID", "entity 2 mention ID", 'entity 1 type', 'entity 2 type']][file_tagged_truth['rel type'] == rel]
                    
                    if entities_type.empty == False:
                        snowball_entity = pos_entity_type_list(entities_type)
                        snowball_pos_ent_type[rel].update(snowball_entity)
                        
                    snowball_index, empty_ct, sum_ct = cal_pos_pattern_count(snowball_index, pos_pattern_list)
                    list_empty_ct.append(empty_ct)
                    list_sum_ct.append(sum_ct)
                    snowball_index_rel[rel].update(snowball_index)
            except EmptyDataError:
                print(f"No columns to parse from file {file}")
            
    return snowball_index_rel, snowball_pos_ent_type, rel_type_out, list_empty_ct, list_sum_ct

In [31]:
def pos_entity_type_list(entities_type):
    
    ground_truth_id = entities_type[["entity 1 mention ID", "entity 2 mention ID"]].values.tolist()
    entities_type = entities_type[['entity 1 type', 'entity 2 type']].values.tolist()
    
    snowball_entity = set()
    
    for index, ids in enumerate(ground_truth_id):
        gt_dict = {}
        gt_dict[ids[0]] = entities_type[index][0]
        gt_dict[ids[1]] = entities_type[index][1]
        
        gt_dict_keys = sort_entity_ids([ids])
        
        entity1 = gt_dict[gt_dict_keys[0][0]]
        entity2 = gt_dict[gt_dict_keys[0][1]]
        
        if entity1 == entity1 and entity2 == entity2:
            tmp = entity1 + "-" + entity2
            snowball_entity.add(tmp)
    
    # snowball_entity = set()
    # for index, pos_pattern in enumerate(pos_pattern_list):
    #     tmp = entity1[index] + "-" + entity2[index]
    #     print(tmp)
    #     snowball_entity.add(tmp)
        # if pos_pattern in snowball_entity:
        #     tmp = entity1[index] + "-" + entity2[index]
        #     # tmp_set = snowball_entity[pos_pattern]
        #     snowball_entity.add(tmp)
        #     # print(pos_pattern, "-->", tmp)
        #     # snowball_entity[pos_pattern] = tmp_set
        # elif pos_pattern != pos_pattern:
        #     pass
        # else:
        #     tmp = entity1[index] + "-" + entity2[index]
        #     # print(pos_pattern, "==>", tmp)
        #     snowball_entity = {tmp}
        #     # snowball_entity[pos_pattern] = {tmp}
    
    return snowball_entity

In [32]:
def cal_rank(snowball_index):
    
    snowball_pos_pattern = {}
    total_index = 0
    
    for value in snowball_index.values():
        total_index += value
    
    for key, value in snowball_index.items():
        snowball_pos_pattern[key] = value/total_index
    
    return snowball_pos_pattern, total_index
    

In [33]:
def sort_dict(snowball_pos_pattern):
    return dict(sorted(snowball_pos_pattern.items(), key=operator.itemgetter(1), reverse=True))

In [34]:
def remove_out(snowball_pos_pattern):
    snowball_pos_pattern_values = np.array(list(snowball_pos_pattern.values()))
    snowball_pos_pattern_keys = list(snowball_pos_pattern.keys())
    
    mean = np.mean(snowball_pos_pattern_values, axis=0)
    sd = np.std(snowball_pos_pattern_values, axis=0)
    
    final_list = [x for x in snowball_pos_pattern_values if (x >= mean - 2 * sd)]
    final_list = [x for x in final_list if (x <= mean + 2 * sd)]
    
    snowball_pos_pattern_keys = snowball_pos_pattern_keys[:len(final_list)]
    
    snowball_pos_pattern_filter = dict(zip(snowball_pos_pattern_keys, final_list))
    
    return snowball_pos_pattern_filter
    

## Find Valid POS Pattern

In [35]:
def re_rank_snowball(snowball_pos_pattern, snowball_key, total_index):
    
    for key in snowball_pos_pattern.keys():
        
        if snowball_key == key:
            snowball_pos_pattern[key] = ((snowball_pos_pattern[key]*(total_index)) + 1)/(total_index + 1)
        else:
            snowball_pos_pattern[key] = (snowball_pos_pattern[key]*(total_index))/(total_index + 1)
            
    return snowball_pos_pattern
    

In [36]:
def check_pos_pattern_and_update(snowball_pos_pattern, pos_pattern, total_index, re_compute = False):
    
    if pos_pattern in snowball_pos_pattern.keys():
        score = snowball_pos_pattern[pos_pattern]
        if re_compute == True:
            print("\n=====Matched with SnowBall -- Re-Computing Snowball=====\n")
            snowball_pos_pattern = re_rank_snowball(snowball_pos_pattern, pos_pattern, total_index)
        return score, snowball_pos_pattern
    else:
        return False, snowball_pos_pattern

In [37]:
def check_rel_pos_pattern_and_update(snowball_pos_pattern_filter_rel, pos_pattern, filter_rel_types, total_index_rel, re_compute = False):
    
    pos_len = len(snowball_pos_pattern_filter_rel) - 1
    max_score = -1
    rel_type = ''

    for index, (snowball_pos_pattern_key, snowball_pos_pattern_value) in enumerate(snowball_pos_pattern_filter_rel.items()):
        # print("==>", snowball_pos_pattern_key, filter_rel_types)
        if snowball_pos_pattern_key in filter_rel_types:
            if pos_pattern in snowball_pos_pattern_value.keys():
                score = snowball_pos_pattern_value[pos_pattern]
                # print("==>", snowball_pos_pattern_key, pos_pattern, score)

                if score > max_score:
                    max_score = score
                    rel_type = snowball_pos_pattern_key

                if re_compute == True and index == pos_len:
                    print("\n=====Matched with SnowBall -- Re-Computing Snowball=====\n")
                    snowball_pos_pattern_filter_rel[rel_type] = re_rank_snowball(snowball_pos_pattern_filter_rel[rel_type], pos_pattern, total_index_rel[rel_type])
                
    if max_score != -1:
        return max_score, rel_type, snowball_pos_pattern_filter_rel
    else:
        return False, False, snowball_pos_pattern_filter_rel

In [38]:
def filter_valid_rel_type(snowball_pos_ent_type, entities_type):
    
    filter_rel_types = []
    for key, value in snowball_pos_ent_type.items():
        if entities_type in value:
            filter_rel_types.append(key)
            
    return filter_rel_types
    

In [39]:
#Global snowball

ACE2004_file_dir_files_sample = ACE2004_file_dir_files[:int(len(ACE2004_file_dir_files) * .70)]
# snowball_index = generate_snowball(ACE2004_file_dir_files_sample)
# snowball_pos_pattern, total_index = cal_rank(snowball_index)
# snowball_pos_pattern = sort_dict(snowball_pos_pattern)
# snowball_pos_pattern_filter = remove_out(snowball_pos_pattern)

In [40]:
#Relation focused snowball

snowball_index_rel, snowball_pos_ent_type, rel_type_out, list_empty_ct, list_sum_ct = generate_rel_snowball(ACE2004_file_dir_files_sample, 10)

total_index_rel = {}
snowball_pos_pattern_filter_rel = {}

for key, value in snowball_index_rel.items():
    snowball_pos_pattern, total_index_rel[key] = cal_rank(value)
    snowball_pos_pattern = sort_dict(snowball_pos_pattern)
    snowball_pos_pattern_filter_rel[key] = remove_out(snowball_pos_pattern)

No columns to parse from file ../Data/relex_processed_data/relex/ACE2004/ground_truth\CNN20001014.1400.0553.csv
No columns to parse from file ../Data/relex_processed_data/relex/ACE2004/ground_truth\CNN20001014.1400.0553.csv
No columns to parse from file ../Data/relex_processed_data/relex/ACE2004/ground_truth\CNN20001014.1400.0553.csv
No columns to parse from file ../Data/relex_processed_data/relex/ACE2004/ground_truth\CNN20001014.1400.0553.csv


In [41]:
#Global snowball

valid_relationships = []
valid_ids = []

for index, entity_list in enumerate(sentences_entity_list):
    entities_relations = find_relation(tagged_dict, entity_list[0], entity_list)
    print(ACE2004_54_text_list[entity_list[0]])
    print(entity_list[1])
    print("+++++")
    for entity_relation in entities_relations:
        print(entity_relation)
        score, snowball_pos_pattern = check_pos_pattern_and_update(snowball_pos_pattern_filter, entity_relation[2][1], total_index)
        print("POS Pattern Score: ", score)
        if score != False:
            valid_relationships.append(entity_relation[2][0])
            valid_ids.append(entity_relation[1])
    print("=====")
    print("\n")
print(valid_relationships)
print(valid_ids)

Harare 7-15 (AFP)-
[['Harare', 'AFP)-'], [['19-1', '0'], ['20-29', '5']], ['GPE', 'ORG']]
+++++
[['Harare', 'AFP)-'], ['19-1', '20-29', 'GPE-ORG'], ['Harare 7 - 15 ( AFP)-', 'NUM-SYM-NUM-PUNCT']]


NameError: name 'snowball_pos_pattern_filter' is not defined

In [42]:
#Relation focused snowball

valid_relationships = []
valid_ids = []
valid_pos = []
valid_rel = []
valid_ents = []
valid_ents_typ = []

for index, entity_list in enumerate(sentences_entity_list):
    entities_relations = find_relation(tagged_dict, entity_list[0], entity_list)
    print(ACE2004_54_text_list[entity_list[0]])
    print(entity_list[1])
    print("+++++")
    for entity_relation in entities_relations:
        print(entity_relation)
        filter_rel_types = filter_valid_rel_type(snowball_pos_ent_type, entity_relation[1][2])
        score, rel_type, snowball_pos_pattern_filter_rel = check_rel_pos_pattern_and_update(snowball_pos_pattern_filter_rel, entity_relation[2][1], filter_rel_types, total_index_rel)
        print("POS Pattern Score: ", score)
        print("Relationship Type: ", rel_type)
        if score != False:
            valid_ents.append([entity_relation[0][0], entity_relation[0][1]])
            valid_ents_typ.append(entity_relation[1][2])
            valid_relationships.append(entity_relation[2][0])
            valid_pos.append(entity_relation[2][1])
            valid_rel.append(rel_type)
            valid_ids.append(entity_relation[1][:2])
    print("=====")
    print("\n")

print(valid_ents)
print("\n")
print(valid_ents_typ)
print("\n")
print(valid_relationships)
print("\n")
print(valid_pos)
print("\n")
print(valid_rel)
print("\n")
print(valid_ids)

Harare 7-15 (AFP)-
[['Harare', 'AFP)-'], [['19-1', '0'], ['20-29', '5']], ['GPE', 'ORG']]
+++++
[['Harare', 'AFP)-'], ['19-1', '20-29', 'GPE-ORG'], ['Harare 7 - 15 ( AFP)-', 'NUM-SYM-NUM-PUNCT']]
POS Pattern Score:  False
Relationship Type:  False
=====


Zimbabwe's President Robert Mugabe announced to journalists this evening Saturday that he had formed a new government, following legislative elections last June, with no opposition members included.
[['Zimbabwe', 'President', 'Robert Mugabe', 'journalists', 'he', 'government', 'members'], [['21-30', '0'], ['22-34', '2'], ['22-35', '3'], ['25-39', '7'], ['22-36', '12'], ['26-40', '17'], ['27-41', '28']], ['GPE', 'PER', 'PER', 'PER', 'PER', 'ORG', 'PER']]
+++++
[['Zimbabwe', 'President'], ['21-30', '22-34', 'GPE-PER'], ["Zimbabwe 's President", 'PART']]
POS Pattern Score:  0.017553191489361703
Relationship Type:  EMP-ORG
[['President', 'Robert Mugabe'], ['22-34', '22-35', 'PER-PER'], ['President  Robert Mugabe', '']]
POS Pattern Score: 

In [44]:
valid_ids

[['21-30', '22-34'],
 ['22-36', '26-40'],
 ['1-4', '1-5'],
 ['1-5', '1-6'],
 ['1-6', '28-2'],
 ['26-7', '3-8'],
 ['5-11', '6-12'],
 ['7-13', '7-14'],
 ['7-15', '9-16'],
 ['12-21', '12-20'],
 ['14-23', '21-32'],
 ['21-32', '15-24']]

# Benchmarking

In [ ]:
def benchmark(valid_relationships, valid_ids, valid_rel, ground_truth, rel_type_out):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    valid_relationships = list(map(str, valid_relationships))
    ground_truth = ground_truth[ground_truth["pos_pattern"].notna()]
    ground_truth_relationships = list(map(str, ground_truth["span"].tolist()))
    ground_truth_relationships_type = list(map(str, ground_truth["rel type"].tolist()))
    
    ground_truth_id = ground_truth[["entity 1 mention ID", "entity 2 mention ID"]].values.tolist()
    ground_truth_id = sort_entity_ids(ground_truth_id)
    valid_fixed_ids = []
    for ids in valid_ids:
        valid_fixed_ids.append(ids[:2])
    valid_ids = sort_entity_ids(valid_fixed_ids)
    pred_rel = ''
    ground_truth_rel = ''
    
    print("\n")
    print("DETECTED RELATIONSHIPS: ")
    [print(relationships) for relationships in valid_relationships]
    print("\n")
    print("GROUND TRUTH RELATIONSHIPS: ")
    [print(relationships) for relationships in ground_truth_relationships]
    print("\n")
    
#     for relationship in valid_relationships:
#         if df.get_close_matches(relationship, ground_truth, cutoff=.8):
#             print("Matched Relationship: ", df.get_close_matches(relationship, ground_truth, cutoff=.8)[0])
#             true_positives += 1
#         else:
#             false_positives += 1
    
#     for relationship in ground_truth:
#         if len(df.get_close_matches(relationship, valid_relationships, cutoff=.8)):
#             false_negatives += 1

    pred_grd_rel_type = []
    for index, ids in enumerate(valid_ids):
        if ids in ground_truth_id:
            if len(valid_rel) != 0:
                id_index = ground_truth_id.index(ids)
                ground_truth_rel = ground_truth_relationships_type[id_index]
                pred_rel = valid_rel[index]
                pred_grd_rel_type.append([pred_rel, ground_truth_rel])
                if ground_truth_rel not in rel_type_out:
                    if ground_truth_rel == pred_rel:
                        print("\nMatched Relationship: %s with relationship %s" %(valid_relationships[index], pred_rel))
                        true_positives += 1
                    else:
                        print("\nIndex matched for ''%s'' but relationship type not matched with PREDICTED as: %s and ACTUAL as: %s" %(valid_relationships[index], pred_rel, ground_truth_rel))
                        false_positives += 1
                else:
                    print("Ignoring less significant relationalship type ''%s''" %(ground_truth_rel))
            else:
                print("\nMatched Relationship: %s" %(valid_relationships[index]))
                true_positives += 1
        else:
            pred_grd_rel_type.append([valid_rel[index], 'NA'])
            false_positives += 1
    
    for index, ids in enumerate(ground_truth_id):
        if ids not in valid_ids:
            false_negatives += 1
            
    return true_positives, false_positives, false_negatives, pred_grd_rel_type

In [ ]:
true_positives, false_positives, false_negatives, _ = benchmark(valid_relationships, valid_ids, [], ACE2004_54_truth[["rel type", "entity 1 mention ID", "entity 2 mention ID", "pos_pattern", "span"]], rel_type_out)

In [ ]:
def precision(true_positives, false_positives):
    print("Precision: ", true_positives/(true_positives + false_positives))
    return true_positives/(true_positives + false_positives)

In [ ]:
def recall(true_positives, false_negatives):
    print("Recall: ", true_positives/(true_positives + false_negatives))
    return true_positives/(true_positives + false_negatives)

In [ ]:
true_positives, false_positives, false_negatives

In [ ]:
precision(true_positives, false_positives)

In [ ]:
recall(true_positives, false_negatives)

In [ ]:
def benchmark_dataset(tagged_token_files, ground_truth_files, raw_text_files, snowball_pos_pattern, total_index, snowball_pos_ent_type = [], rel_type_out = [], re_compute = False, rel_snowball = False):
    
    all_precision = []
    all_recall = []
    
    all_valid_relationships = []
    all_valid_pos = []
    all_valid_rel = []
    all_valid_ents = []
    all_valid_ents_type = []
    all_pred_grd_rel_type = []
    
    for index, file in enumerate(tagged_token_files):
        
        try: 
            print("==================Start of File %d==================" % index)
            print("Reading Files: " + raw_text_files[index] + "\n- " + file + "\n- " + ground_truth_files[index])
            print("\n")
            tagged_token = pd.read_csv(file, sep = "\t")
            ground_truth = pd.read_csv(ground_truth_files[index], sep = "\t")
            raw_text = open(raw_text_files[index], "r", encoding="utf8")
            raw_text_list = raw_text.readlines()
            raw_text_list = [sentence for sentence in raw_text_list if sentence != '\n']
            raw_text_list = [sentence.replace(' \n', '') for sentence in raw_text_list]
            
            tagged_token_dict = create_tagged_dict(tagged_token)

            sentences_entity_list = []

            for index, sentence in enumerate(raw_text_list):
                sentence_nlp = nlp(sentence)
                
                entities, indices, entities_types = sentence_entity(tagged_token_dict, index, sentence_nlp, False)
                # sent_roots, sent_roots_index = extract_root(tagged_token_dict, index, sentence_nlp)
                sentences_entity_list.append([index, [entities, indices, entities_types]])
                
            # sentences_entity_list = filter_sentences(sentences_entity_list)

            valid_relationships = []
            valid_ids = []
            valid_pos = []
            valid_rel = []
            valid_ents = []
            valid_ents_typ = []

            for index, entity_list in enumerate(sentences_entity_list):
                entities_relations = find_relation(tagged_token_dict, entity_list[0], entity_list)
                print(raw_text_list[entity_list[0]])
                # print(entity_list[1])
                # print("+++++")
                for entity_relation in entities_relations:
                    # print(entity_relation)
                    if rel_snowball == False:
                        score, snowball_pos_pattern = check_pos_pattern_and_update(snowball_pos_pattern, entity_relation[2][1], total_index, re_compute)
                    else:
                        filter_rel_types = filter_valid_rel_type(snowball_pos_ent_type, entity_relation[1][2])
                        score, rel_type, snowball_pos_pattern_filter_rel = check_rel_pos_pattern_and_update(snowball_pos_pattern, entity_relation[2][1], filter_rel_types, total_index, re_compute)
                    # print("POS Pattern Score: ", score)
                    if score != False:
                        valid_ents.append([entity_relation[0][0], entity_relation[0][1]])
                        valid_ents_typ.append(entity_relation[1][2])
                        valid_relationships.append(entity_relation[2][0])
                        valid_pos.append(entity_relation[2][1])
                        valid_ids.append(entity_relation[1])
                        if rel_snowball == True:
                            valid_rel.append(rel_type)
                            
                # print("=====")
            

            if "span" in ground_truth:
                true_positives, false_positives, false_negatives, pred_grd_rel_type = benchmark(valid_relationships, valid_ids, valid_rel, ground_truth[["rel type", "entity 1 mention ID", "entity 2 mention ID", "pos_pattern", "span"]], rel_type_out)

                print("\n")
                print("BENCHMARK:")
                print(len(pred_grd_rel_type))

                print("True Positives: %d" % true_positives)
                print("False Positives: %d" % false_positives)
                print("False Negatives: %d" % false_negatives)

                print("\n")
                
                all_valid_relationships.append(valid_relationships)
                all_valid_pos.append(valid_pos)
                all_valid_rel.append(valid_rel)
                all_valid_ents.append(valid_ents)
                all_valid_ents_type.append(valid_ents_typ)
                all_pred_grd_rel_type.append(pred_grd_rel_type)

                if((true_positives + false_positives) != 0):
                    all_precision.append(precision(true_positives, false_positives))
                else:
                     all_precision.append(0)

                if((true_positives + false_negatives) != 0):
                    all_recall.append(recall(true_positives, false_negatives))
                else:
                    all_recall.append(0)
                    
            else:
                print("\nCan't benchmark ground truth file empty")


            print("==================End of File==================")

            print("\n")
        
        except EmptyDataError:
            print("One of the csv file is empty")
        
    if(len(all_precision) > 0 and len(all_recall) > 0):
        
        print("==================Final Result==================")

        print("\n")

        print("Average Precision %f" %np.mean(all_precision))
        print("Average Recall %f" %np.mean(all_recall))
        
        return all_precision, all_recall, all_valid_relationships, all_valid_pos, all_valid_rel, all_valid_ents, all_valid_ents_type, all_pred_grd_rel_type

        print("\n")
        
        print("===============================================")

In [ ]:
columns = ["File Name", "Precision", "Recall"]

# Benchmarking ACE2004 dataset

In [ ]:
tagged_token_files_2004 = glob.glob("../Data/relex_processed_data/relex/ACE2004/tagged_tokens/*")
ground_truth_files_2004 = glob.glob("../Data/relex_processed_data/relex/ACE2004/ground_truth/*")
raw_text_files_2004 = glob.glob("../Data/relex_processed_data/relex/ACE2004/raw_text/*")

## 70% Sampling of all files

In [ ]:
tagged_token_files_2004_sample = tagged_token_files_2004[:int(len(tagged_token_files_2004) * .70)]
ground_truth_files_2004_sample = ground_truth_files_2004[:int(len(ground_truth_files_2004) * .70)]
raw_text_files_2004_sample = raw_text_files_2004[:int(len(raw_text_files_2004) * .70)]

## 30% Sampling of all files for testing

In [ ]:
tagged_token_files_2004_sample_test = tagged_token_files_2004[int(len(tagged_token_files_2004) * .70):]
ground_truth_files_2004_sample_test = ground_truth_files_2004[int(len(ground_truth_files_2004) * .70):]
raw_text_files_2004_sample_test = raw_text_files_2004[int(len(raw_text_files_2004) * .70):]

## 70% sampled SnowBall from ground truth

In [ ]:
ACE2004_file_dir_files = glob.glob("../Data/relex_processed_data/relex/ACE2004/ground_truth/*.csv")

In [ ]:
ACE2004_file_dir_files_sample = ACE2004_file_dir_files[:int(len(ACE2004_file_dir_files) * .70)]
snowball_index_2004 = generate_snowball(ACE2004_file_dir_files_sample)
snowball_pos_pattern_2004, ACE2004_total_index = cal_rank(snowball_index_2004)
snowball_pos_pattern_2004 = sort_dict(snowball_pos_pattern_2004)
snowball_pos_pattern_2004_filter = remove_out(snowball_pos_pattern_2004)

## 70% sampled SnowBall with relations types from ground truth

In [ ]:
ACE2004_file_dir_files_sample = ACE2004_file_dir_files[:int(len(ACE2004_file_dir_files) * .70)]
snowball_index_rel_2004, snowball_pos_ent_type_2004, rel_type_out_2004, list_empty_ct, sum_ct = generate_rel_snowball(ACE2004_file_dir_files_sample, 10)

print(sum(sum_ct))
print("\nMissing ground truth for %d" %(sum(list_empty_ct)))
ACE2004_total_index_rel = {}
snowball_pos_pattern_2004_filter_rel = {}

for key, value in snowball_index_rel_2004.items():
    snowball_pos_pattern, ACE2004_total_index_rel[key] = cal_rank(value)
    snowball_pos_pattern = sort_dict(snowball_pos_pattern)
    snowball_pos_pattern_2004_filter_rel[key] = remove_out(snowball_pos_pattern)

## Benchmarking on Trained

In [ ]:
all_precision_2004, all_recall_2004 = benchmark_dataset(tagged_token_files_2004_sample, ground_truth_files_2004_sample, raw_text_files_2004_sample, snowball_pos_pattern_2004_filter, ACE2004_total_index)

## Benchmarking on Trained - Rel Snowball

In [ ]:
all_precision_rel_2004, all_recall_rel_2004, all_valid_relationships, all_valid_pos, all_valid_rel, all_valid_ents, all_valid_ents_type, all_pred_grd_rel_type = benchmark_dataset(tagged_token_files_2004_sample, ground_truth_files_2004_sample, raw_text_files_2004_sample, snowball_pos_pattern_2004_filter_rel, ACE2004_total_index_rel, snowball_pos_ent_type_2004, rel_type_out_2004, False, True)

In [ ]:
print('Predicted Rel Type , Actual Rel Type , Correct Prediction , Entity 1 , Entity 2 , Entity 1 type , Entity 2 type , Text Span , Text POS')
for index1, doc_enitites in enumerate(all_valid_ents):
    for index2, entities in enumerate(doc_enitites):
        # tmp1 = tmp2 = 'test'
        tmp1, tmp2 = all_pred_grd_rel_type[index1][index2]
        if tmp1==tmp2:
            correct = True
        else:
            correct = False
        print(tmp1, ',', tmp2, ',', correct, ',', entities[0], ',', entities[1], ',', all_valid_ents_type[index1][index2].split('-')[0], ',', all_valid_ents_type[index1][index2].split('-')[1], ',', all_valid_relationships[index1][index2], ',', all_valid_pos[index1][index2])

## Benchmarking on Test

In [ ]:
all_precision_2004_test, all_recall_2004_test = benchmark_dataset(tagged_token_files_2004_sample_test, ground_truth_files_2004_sample_test, raw_text_files_2004_sample_test, snowball_pos_pattern_2004_filter, ACE2004_total_index, True)

## Benchmarking on Test - Rel Snowball

In [ ]:
all_precision_rel_2004_test, all_recall_rel_2004_test, all_valid_relationships, all_valid_pos, all_valid_rel, all_valid_ents, all_valid_ents_type, all_pred_grd_rel_type = benchmark_dataset(tagged_token_files_2004_sample_test, ground_truth_files_2004_sample_test, raw_text_files_2004_sample_test, snowball_pos_pattern_2004_filter_rel, ACE2004_total_index_rel, snowball_pos_ent_type_2004, rel_type_out_2004, True, True)

In [ ]:
# print('Predicted Rel Type , Actual Rel Type , Correct Prediction , Entity 1 , Entity 2 , Entity 1 type , Entity 2 type , Text Span , Text POS')
for index1, doc_enitites in enumerate(all_valid_ents):
    for index2, entities in enumerate(doc_enitites):
        # tmp1 = tmp2 = 'test'
        tmp1, tmp2 = all_pred_grd_rel_type[index1][index2]
        if tmp1==tmp2:
            correct = True
        else:
            correct = False
        print(tmp1, ',', tmp2, ',', correct, ',', entities[0], ',', entities[1], ',', all_valid_ents_type[index1][index2].split('-')[0], ',', all_valid_ents_type[index1][index2].split('-')[1], ',', all_valid_relationships[index1][index2], ',', all_valid_pos[index1][index2])

## Making result dataframe

In [ ]:
ACE2004_sample_result = pd.DataFrame(data=[[x.split('/')[-1].split("\\")[-1] for x in raw_text_files_2004_sample], all_precision_rel_2004, all_recall_rel_2004]).T
ACE2004_sample_result.columns = columns

ACE2004_test_result = pd.DataFrame(data=[[x.split('/')[-1].split("\\")[-1] for x in raw_text_files_2004_sample_test], all_precision_rel_2004_test, all_recall_rel_2004_test]).T
ACE2004_test_result.columns = columns

In [ ]:
ACE2004_results = pd.ExcelWriter('Rel_Type/ACE2004_rel_results.xlsx', engine='xlsxwriter') 

ACE2004_sample_result.to_excel(ACE2004_results, sheet_name='Sample', index=False)
ACE2004_test_result.to_excel(ACE2004_results, sheet_name='Test', index=False)

ACE2004_results.save()

# Benchmarking ACE2005 dataset

In [ ]:
tagged_token_files_2005 = glob.glob("../Data/relex_processed_data/relex/ACE2005/tagged_tokens/*")
ground_truth_files_2005 = glob.glob("../Data/relex_processed_data/relex/ACE2005/ground_truth/*")
raw_text_files_2005 = glob.glob("../Data/relex_processed_data/relex/ACE2005/raw_text/*")

## 70% Sampling of all files

In [ ]:
tagged_token_files_2005_sample = tagged_token_files_2005[:int(len(tagged_token_files_2005) * .70)]
ground_truth_files_2005_sample = ground_truth_files_2005[:int(len(ground_truth_files_2005) * .70)]
raw_text_files_2005_sample = raw_text_files_2005[:int(len(raw_text_files_2005) * .70)]

## 30% Sampling of all files for testing

In [ ]:
tagged_token_files_2005_sample_test = tagged_token_files_2005[int(len(tagged_token_files_2005) * .70):]
ground_truth_files_2005_sample_test = ground_truth_files_2005[int(len(ground_truth_files_2005) * .70):]
raw_text_files_2005_sample_test = raw_text_files_2005[int(len(raw_text_files_2005) * .70):]

## 70% sampled SnowBall from ground truth

In [ ]:
ACE2005_file_dir_files = glob.glob("../Data/relex_processed_data/relex/ACE2005/ground_truth/*.csv")

In [ ]:
ACE2005_file_dir_files_sample = ACE2005_file_dir_files[:int(len(ACE2005_file_dir_files) * .70)]
snowball_index_2005 = generate_snowball(ACE2005_file_dir_files_sample)
snowball_pos_pattern_2005, ACE2005_total_index = cal_rank(snowball_index_2005)
snowball_pos_pattern_2005 = sort_dict(snowball_pos_pattern_2005)
snowball_pos_pattern_2005_filter = remove_out(snowball_pos_pattern_2005)

## 70% sampled SnowBall with relations types from ground truth

In [ ]:
ACE2005_file_dir_files_sample = ACE2005_file_dir_files[:int(len(ACE2005_file_dir_files) * .70)]
snowball_index_rel_2005, snowball_pos_ent_type_2005, rel_type_out_2005, cmpt_cp, sum_ct = generate_rel_snowball(ACE2005_file_dir_files_sample, 20)

print(sum(sum_ct))
print("\n Missing pos pattern %d" %(sum(cmpt_cp)))
ACE2005_total_index_rel = {}
snowball_pos_pattern_2005_filter_rel = {}

for key, value in snowball_index_rel_2005.items():
    snowball_pos_pattern, ACE2005_total_index_rel[key] = cal_rank(value)
    snowball_pos_pattern = sort_dict(snowball_pos_pattern)
    snowball_pos_pattern_2005_filter_rel[key] = remove_out(snowball_pos_pattern)

## Benchmarking on Trained

In [ ]:
all_precision_2005, all_recall_2005 = benchmark_dataset(tagged_token_files_2005_sample, ground_truth_files_2005_sample, raw_text_files_2005_sample, snowball_pos_pattern_2005_filter, ACE2005_total_index, False)

## Benchmarking on Trained - Rel Snowball

In [ ]:
all_precision_rel_2005, all_recall_rel_2005, all_valid_relationships, all_valid_pos, all_valid_rel, all_valid_ents, all_valid_ents_type, all_pred_grd_rel_type = benchmark_dataset(tagged_token_files_2005_sample, ground_truth_files_2005_sample, raw_text_files_2005_sample, snowball_pos_pattern_2005_filter_rel, ACE2005_total_index_rel, snowball_pos_ent_type_2005, rel_type_out_2005, False, True)

In [ ]:
print('Predicted Rel Type , Actual Rel Type , Correct Prediction , Entity 1 , Entity 2 , Entity 1 type , Entity 2 type , Text Span , Text POS')
for index1, doc_enitites in enumerate(all_valid_ents):
    for index2, entities in enumerate(doc_enitites):
        # tmp1 = tmp2 = 'test'
        tmp1, tmp2 = all_pred_grd_rel_type[index1][index2]
        if tmp1==tmp2:
            correct = True
        else:
            correct = False
        print(tmp1, ',', tmp2, ',', correct, ',', entities[0], ',', entities[1], ',', all_valid_ents_type[index1][index2].split('-')[0], ',', all_valid_ents_type[index1][index2].split('-')[1], ',', all_valid_relationships[index1][index2], ',', all_valid_pos[index1][index2])

## Benchmarking on Test

In [ ]:
all_precision_2005_test, all_recall_2005_test = benchmark_dataset(tagged_token_files_2005_sample_test, ground_truth_files_2005_sample_test, raw_text_files_2005_sample_test, snowball_pos_pattern_2005_filter, ACE2005_total_index, True)

## Benchmarking on Test - Rel Snowball

In [ ]:
all_precision_rel_2005_test, all_recall_rel_2005_test, all_valid_relationships, all_valid_pos, all_valid_rel, all_valid_ents, all_valid_ents_type, all_pred_grd_rel_type = benchmark_dataset(tagged_token_files_2005_sample_test, ground_truth_files_2005_sample_test, raw_text_files_2005_sample_test, snowball_pos_pattern_2005_filter_rel, ACE2005_total_index_rel, snowball_pos_ent_type_2005, rel_type_out_2005, True, True)

In [ ]:
# print('Predicted Rel Type , Actual Rel Type , Correct Prediction , Entity 1 , Entity 2 , Entity 1 type , Entity 2 type , Text Span , Text POS')
for index1, doc_enitites in enumerate(all_valid_ents):
    for index2, entities in enumerate(doc_enitites):
        # tmp1 = tmp2 = 'test'
        tmp1, tmp2 = all_pred_grd_rel_type[index1][index2]
        if tmp1==tmp2:
            correct = True
        else:
            correct = False
        print(tmp1, ',', tmp2, ',', correct, ',', entities[0], ',', entities[1], ',', all_valid_ents_type[index1][index2].split('-')[0], ',', all_valid_ents_type[index1][index2].split('-')[1], ',', all_valid_relationships[index1][index2], ',', all_valid_pos[index1][index2])

## Making result dataframe

In [ ]:
ACE2005_sample_result = pd.DataFrame(data=[[x.split('/')[-1].split("\\")[-1] for x in raw_text_files_2005_sample], all_precision_rel_2005, all_recall_rel_2005]).T
ACE2005_sample_result.columns = columns

ACE2005_test_result = pd.DataFrame(data=[[x.split('/')[-1].split("\\")[-1] for x in raw_text_files_2005_sample_test], all_precision_rel_2005_test, all_recall_rel_2005_test]).T
ACE2005_test_result.columns = columns

In [ ]:
ACE2005_results = pd.ExcelWriter('Rel_Type/ACE2005_rel_results.xlsx', engine='xlsxwriter') 

ACE2005_sample_result.to_excel(ACE2005_results, sheet_name='Sample', index=False)
ACE2005_test_result.to_excel(ACE2005_results, sheet_name='Test', index=False)

ACE2005_results.save()

# Benchmarking docred dataset

In [ ]:
tagged_token_files_docred = glob.glob("../Data/relex_processed_data/relex/docred/tagged_tokens/*")
ground_truth_files_docred = glob.glob("../Data/relex_processed_data/relex/docred/ground_truth/*")
raw_text_files_docred = glob.glob("../Data/relex_processed_data/relex/docred/raw_text/*")

## 60% Sampling of all files

In [ ]:
tagged_token_files_docred_sample = tagged_token_files_docred[:int(len(tagged_token_files_docred) * .60)]
ground_truth_files_docred_sample = ground_truth_files_docred[:int(len(ground_truth_files_docred) * .60)]
raw_text_files_docred_sample = raw_text_files_docred[:int(len(raw_text_files_docred) * .60)]

## 40% Sampling of all files for testing

In [ ]:
tagged_token_files_docred_sample_test = tagged_token_files_docred[int(len(tagged_token_files_docred) * .60):]
ground_truth_files_docred_sample_test = ground_truth_files_docred[int(len(ground_truth_files_docred) * .60):]
raw_text_files_docred_sample_test = raw_text_files_docred[int(len(raw_text_files_docred) * .60):]

## 70% sampled SnowBall from ground truth

In [ ]:
docred_file_dir_files = glob.glob("../Data/relex_processed_data/relex/docred/ground_truth/*.csv")

In [ ]:
docred_file_dir_files_sample = docred_file_dir_files[:int(len(docred_file_dir_files) * .70)]
snowball_index_docred = generate_snowball(docred_file_dir_files_sample)
snowball_pos_pattern_docred, docred_total_index  = cal_rank(snowball_index_docred)
snowball_pos_pattern_docred = sort_dict(snowball_pos_pattern_docred)
snowball_pos_pattern_docred_filter = remove_out(snowball_pos_pattern_docred)

## 70% sampled SnowBall with relations types from ground truth

In [ ]:
docred_file_dir_files_sample = docred_file_dir_files[:int(len(docred_file_dir_files) * .70)]
snowball_index_rel_docred, snowball_pos_ent_type_docred, rel_type_out_docred, empt_ct, sum_ct = generate_rel_snowball(docred_file_dir_files_sample, 5)

print(sum(sum_ct))
print("\nMissing pos pattern %d" %(sum(empt_ct)))
docred_total_index_rel = {}
snowball_pos_pattern_docred_filter_rel = {}

for key, value in snowball_index_rel_docred.items():
    snowball_pos_pattern, docred_total_index_rel[key] = cal_rank(value)
    snowball_pos_pattern = sort_dict(snowball_pos_pattern)
    snowball_pos_pattern_docred_filter_rel[key] = remove_out(snowball_pos_pattern)

## Benchmarking on Trained

In [ ]:
all_precision_docred, all_recall_docred = benchmark_dataset(tagged_token_files_docred_sample, ground_truth_files_docred_sample, raw_text_files_docred_sample, snowball_pos_pattern_docred_filter, docred_total_index, False)

## Benchmarking on Trained - Rel Snowball

In [ ]:
all_precision_rel_docred, all_recall_rel_docred, all_valid_relationships, all_valid_pos, all_valid_rel, all_valid_ents, all_valid_ents_type, all_pred_grd_rel_type = benchmark_dataset(tagged_token_files_docred_sample, ground_truth_files_docred_sample, raw_text_files_docred_sample, snowball_pos_pattern_docred_filter_rel, docred_total_index_rel, snowball_pos_ent_type_docred, rel_type_out_docred, False, True)

In [ ]:
print('Predicted Rel Type , Actual Rel Type , Correct Prediction , Entity 1 , Entity 2 , Entity 1 type , Entity 2 type , Text Span , Text POS')
for index1, doc_enitites in enumerate(all_valid_ents):
    for index2, entities in enumerate(doc_enitites):
        # tmp1 = tmp2 = 'test'
        tmp1, tmp2 = all_pred_grd_rel_type[index1][index2]
        if tmp1==tmp2:
            correct = True
        else:
            correct = False
        print(tmp1, ',', tmp2, ',', correct, ',', entities[0], ',', entities[1], ',', all_valid_ents_type[index1][index2].split('-')[0], ',', all_valid_ents_type[index1][index2].split('-')[1], ',', all_valid_relationships[index1][index2], ',', all_valid_pos[index1][index2])

## Benchmarking on Test

In [ ]:
all_precision_docred_test, all_recall_docred_test = benchmark_dataset(tagged_token_files_docred_sample_test, ground_truth_files_docred_sample_test, raw_text_files_docred_sample_test, snowball_pos_pattern_docred_filter, docred_total_index, True)

## Benchmarking on Test - Rel Snowball

In [ ]:
all_precision_rel_docred_test, all_recall_rel_docred_test, all_valid_relationships, all_valid_pos, all_valid_rel, all_valid_ents, all_valid_ents_type, all_pred_grd_rel_type = benchmark_dataset(tagged_token_files_docred_sample_test, ground_truth_files_docred_sample_test, raw_text_files_docred_sample_test, snowball_pos_pattern_docred_filter_rel, docred_total_index_rel, snowball_pos_ent_type_docred, rel_type_out_docred, True, True)

In [ ]:
# print('Predicted Rel Type , Actual Rel Type , Correct Prediction , Entity 1 , Entity 2 , Entity 1 type , Entity 2 type , Text Span , Text POS')
for index1, doc_enitites in enumerate(all_valid_ents):
    for index2, entities in enumerate(doc_enitites):
        # tmp1 = tmp2 = 'test'
        tmp1, tmp2 = all_pred_grd_rel_type[index1][index2]
        if tmp1==tmp2:
            correct = True
        else:
            correct = False
        print(tmp1, ',', tmp2, ',', correct, ',', entities[0], ',', entities[1], ',', all_valid_ents_type[index1][index2].split('-')[0], ',', all_valid_ents_type[index1][index2].split('-')[1], ',', all_valid_relationships[index1][index2], ',', all_valid_pos[index1][index2])

## Making result dataframe

In [ ]:
docred_sample_result = pd.DataFrame(data=[[x.split('/')[-1].split("\\")[-1] for x in raw_text_files_docred_sample], all_precision_rel_docred, all_recall_rel_docred]).T
docred_sample_result.columns = columns

docred_test_result = pd.DataFrame(data=[[x.split('/')[-1].split("\\")[-1] for x in raw_text_files_docred_sample_test], all_precision_rel_docred_test, all_recall_rel_docred_test]).T
docred_test_result.columns = columns

In [ ]:
docred_results = pd.ExcelWriter('Rel_Type/docred_rel_results.xlsx', engine='xlsxwriter') 

docred_sample_result.to_excel(docred_results, sheet_name='Sample', index=False)
docred_test_result.to_excel(docred_results, sheet_name='Test', index=False)

docred_results.save()

# Benchmarking re3d dataset

In [ ]:
tagged_token_files_re3d = glob.glob("../Data/relex_processed_data/relex/re3d/tagged_tokens/*")
ground_truth_files_re3d = glob.glob("../Data/relex_processed_data/relex/re3d/ground_truth/*")
raw_text_files_re3d = glob.glob("../Data/relex_processed_data/relex/re3d/raw_text/*")

## 70% Sampling of all files

In [ ]:
tagged_token_files_re3d_sample = tagged_token_files_re3d[:int(len(tagged_token_files_re3d) * .70)]
ground_truth_files_re3d_sample = ground_truth_files_re3d[:int(len(ground_truth_files_re3d) * .70)]
raw_text_files_re3d_sample = raw_text_files_re3d[:int(len(raw_text_files_re3d) * .70)]

## 30% Sampling of all files for testing

In [ ]:
tagged_token_files_re3d_sample_test = tagged_token_files_re3d[int(len(tagged_token_files_re3d) * .70):]
ground_truth_files_re3d_sample_test = ground_truth_files_re3d[int(len(ground_truth_files_re3d) * .70):]
raw_text_files_re3d_sample_test = raw_text_files_re3d[int(len(raw_text_files_re3d) * .70):]

## 70% sampled SnowBall from ground truth

In [ ]:
re3d_file_dir_files = glob.glob("../Data/relex_processed_data/relex/re3d/ground_truth/*.csv")

In [ ]:
re3d_file_dir_files_sample = re3d_file_dir_files[:int(len(re3d_file_dir_files) * .70)]
snowball_index_re3d = generate_snowball(re3d_file_dir_files_sample)
snowball_pos_pattern_re3d, re3d_total_index = cal_rank(snowball_index_re3d)
snowball_pos_pattern_re3d = sort_dict(snowball_pos_pattern_re3d)
snowball_pos_pattern_re3d_filter = remove_out(snowball_pos_pattern_re3d)

## 70% sampled SnowBall with relations types from ground truth

In [ ]:
re3d_file_dir_files_sample = re3d_file_dir_files[:int(len(re3d_file_dir_files) * .70)]
snowball_index_rel_re3d, snowball_pos_ent_type_re3d, rel_type_out_re3d, emp_ct = generate_rel_snowball(re3d_file_dir_files_sample, 10)

print("\nMissing pos pattern %d" %(sum(emp_ct)))
re3d_total_index_rel = {}
snowball_pos_pattern_re3d_filter_rel = {}

for key, value in snowball_index_rel_re3d.items():
    snowball_pos_pattern, re3d_total_index_rel[key] = cal_rank(value)
    snowball_pos_pattern = sort_dict(snowball_pos_pattern)
    snowball_pos_pattern_re3d_filter_rel[key] = remove_out(snowball_pos_pattern)

## Benchmarking on Trained

In [ ]:
all_precision_re3d, all_recall_re3d = benchmark_dataset(tagged_token_files_re3d_sample, ground_truth_files_re3d_sample, raw_text_files_re3d_sample, snowball_pos_pattern_re3d_filter, re3d_total_index, False)

## Benchmarking on Trained - Rel Snowball

In [ ]:
all_precision_rel_re3d, all_recall_rel_re3d, all_valid_relationships, all_valid_pos, all_valid_rel, all_valid_ents, all_valid_ents_type, all_pred_grd_rel_type = benchmark_dataset(tagged_token_files_re3d_sample, ground_truth_files_re3d_sample, raw_text_files_re3d_sample, snowball_pos_pattern_re3d_filter_rel, re3d_total_index_rel, snowball_pos_ent_type_re3d, rel_type_out_re3d, False, True)

In [ ]:
print('Predicted Rel Type , Actual Rel Type , Correct Prediction , Entity 1 , Entity 2 , Entity 1 type , Entity 2 type , Text Span , Text POS')
for index1, doc_enitites in enumerate(all_valid_ents):
    for index2, entities in enumerate(doc_enitites):
        # tmp1 = tmp2 = 'test'
        tmp1, tmp2 = all_pred_grd_rel_type[index1][index2]
        if tmp1==tmp2:
            correct = True
        else:
            correct = False
        print(tmp1, ',', tmp2, ',', correct, ',', entities[0], ',', entities[1], ',', all_valid_ents_type[index1][index2].split('-')[0], ',', all_valid_ents_type[index1][index2].split('-')[1], ',', all_valid_relationships[index1][index2], ',', all_valid_pos[index1][index2])

## Benchmarking on Test

In [ ]:
all_precision_re3d_test, all_recall_re3d_test = benchmark_dataset(tagged_token_files_re3d_sample_test, ground_truth_files_re3d_sample_test, raw_text_files_re3d_sample_test, snowball_pos_pattern_re3d_filter, re3d_total_index, True)

## Benchmarking on Test - Rel Snowball

In [ ]:
all_precision_rel_re3d_test, all_recall_rel_re3d_test, all_valid_relationships, all_valid_pos, all_valid_rel, all_valid_ents, all_valid_ents_type, all_pred_grd_rel_type = benchmark_dataset(tagged_token_files_re3d_sample_test, ground_truth_files_re3d_sample_test, raw_text_files_re3d_sample_test, snowball_pos_pattern_re3d_filter_rel, re3d_total_index_rel, snowball_pos_ent_type_re3d, rel_type_out_re3d, True, True)

In [ ]:
# print('Predicted Rel Type , Actual Rel Type , Correct Prediction , Entity 1 , Entity 2 , Entity 1 type , Entity 2 type , Text Span , Text POS')
for index1, doc_enitites in enumerate(all_valid_ents):
    for index2, entities in enumerate(doc_enitites):
        # tmp1 = tmp2 = 'test'
        tmp1, tmp2 = all_pred_grd_rel_type[index1][index2]
        if tmp1==tmp2:
            correct = True
        else:
            correct = False
        print(tmp1, ',', tmp2, ',', correct, ',', entities[0], ',', entities[1], ',', all_valid_ents_type[index1][index2].split('-')[0], ',', all_valid_ents_type[index1][index2].split('-')[1], ',', all_valid_relationships[index1][index2], ',', all_valid_pos[index1][index2])

## Making result dataframe

In [ ]:
re3d_sample_result = pd.DataFrame(data=[[x.split('/')[-1].split("\\")[-1] for x in raw_text_files_re3d_sample], all_precision_rel_re3d, all_recall_rel_re3d]).T
re3d_sample_result.columns = columns

re3d_test_result = pd.DataFrame(data=[[x.split('/')[-1].split("\\")[-1] for x in raw_text_files_re3d_sample_test], all_precision_rel_re3d_test, all_recall_rel_re3d_test]).T
re3d_test_result.columns = columns

In [ ]:
re3d_results = pd.ExcelWriter('REl_Type/re3d_rel_results.xlsx', engine='xlsxwriter') 

re3d_sample_result.to_excel(re3d_results, sheet_name='Sample', index=False)
re3d_test_result.to_excel(re3d_results, sheet_name='Test', index=False)

re3d_results.save()